<a href="https://colab.research.google.com/github/SaumyaG2311/Minor-Project/blob/main/Fake_Review_detection_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
# Keras
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM

In [99]:
import re
import string
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [100]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import files
df = files.upload()

Saving deceptive-opinion.csv to deceptive-opinion (2).csv


In [101]:
import io

In [102]:
data = pd.read_csv(io.BytesIO(df['deceptive-opinion.csv']))

In [103]:
data

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...
...,...,...,...,...,...
1595,deceptive,intercontinental,negative,MTurk,Problems started when I booked the InterContin...
1596,deceptive,amalfi,negative,MTurk,The Amalfi Hotel has a beautiful website and i...
1597,deceptive,intercontinental,negative,MTurk,The Intercontinental Chicago Magnificent Mile ...
1598,deceptive,palmer,negative,MTurk,"The Palmer House Hilton, while it looks good i..."


In [104]:
data = data.drop(["hotel", "polarity","source"], axis=1)

In [105]:
data = data.sample(frac=1)

In [106]:
from sklearn import preprocessing 

# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

# Encode labels in column 'species'. 
data['deceptive'] = label_encoder.fit_transform(data['deceptive']) 

data['deceptive'].unique()

array([0, 1])

In [107]:
#dataset description #truthful=1 deceptive=0
data.groupby('deceptive').describe()

text                                                               
          count unique                                                top freq
deceptive                                                                     
0           800    800  I absolutely loved staying at the Swissotel Ch...    1
1           800    796  Very disappointed in our stay in Chicago Monoc...    2

In [108]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [109]:
# Some preprocesssing that will be common to all the text classification methods

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_char(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x

In [110]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [111]:
data['text'] = data['text'].map(lambda a: clean_numbers(a))

In [112]:
data['text'] = data['text'].map(lambda a: clean_char(a))

In [113]:
data['text'] = data['text'].map(lambda a: clean_text(a))

In [114]:
data['text']

737     top class treatment choos hotel allegro year b...
1324    stay husband homewood suit chicago want someth...
934     locat hotel excel hotel star like say opinion ...
24      check fast room perfect problem standard big c...
615     recent stay talbott hotel chicago famili three...
                              ...                        
754     recent travel chicago busi treat stay intercon...
1593    stay allegro two day big prolem one thing sati...
669     recent stay hotel monaco must say becom one fa...
1376    suppos fun weekend getaway girl chicago turn c...
841     travel often busi hotel rank low list room str...
Name: text, Length: 1600, dtype: object

In [115]:
data.head()

,deceptive,text
737,0,top class treatment choos hotel allegro year b...
1324,0,stay husband homewood suit chicago want someth...
934,1,locat hotel excel hotel star like say opinion ...
24,1,check fast room perfect problem standard big c...
615,0,recent stay talbott hotel chicago famili three...


In [116]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(data['text'])
sequences = tokenizer.texts_to_sequences(data['text'])
dat = pad_sequences(sequences, maxlen=50)

In [117]:
print(dat.shape)

(1600, 50)


In [118]:
#lstm model
model_lstm = Sequential()
model_lstm.add(Embedding(20000, 100, input_length=50))
model_lstm.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(1, activation='sigmoid'))
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [119]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(data['deceptive'])
labels = data['deceptive']

In [120]:
model_lstm.fit(dat, np.array(labels), validation_split=0.3, epochs=3)

Epoch 1/3
35/35 [==============================] - 5s 129ms/step - loss: 0.6840 - accuracy: 0.5598 - val_loss: 0.5757 - val_accuracy: 0.7125
Epoch 2/3
35/35 [==============================] - 4s 113ms/step - loss: 0.4735 - accuracy: 0.7857 - val_loss: 0.4618 - val_accuracy: 0.7958
Epoch 3/3
35/35 [==============================] - 4s 112ms/step - loss: 0.1799 - accuracy: 0.9375 - val_loss: 0.5020 - val_accuracy: 0.8000


In [121]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv

In [122]:
model_conv = create_conv_model()
model_conv.fit(dat, np.array(labels), validation_split=0.3, epochs = 3)

Epoch 1/3
35/35 [==============================] - 2s 57ms/step - loss: 0.6932 - accuracy: 0.5036 - val_loss: 0.6864 - val_accuracy: 0.6375
Epoch 2/3
35/35 [==============================] - 2s 45ms/step - loss: 0.5619 - accuracy: 0.7982 - val_loss: 0.4461 - val_accuracy: 0.8062
Epoch 3/3
35/35 [==============================] - 1s 42ms/step - loss: 0.1947 - accuracy: 0.9339 - val_loss: 0.4552 - val_accuracy: 0.8021


In [123]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True